In [1]:
import pandas as pd
import plotly.express as px

In [2]:
stn = 'd'

In [3]:
sm = pd.read_csv(f'./{stn}/records/sm')

batt = pd.read_csv(f'./{stn}/data/battery')
itemp = pd.read_csv(f'./{stn}/data/itemp')

sm.columns = sm.columns.str.strip()
batt.columns = batt.columns.str.strip()
itemp.columns = itemp.columns.str.strip()

In [4]:
print(f"SM1 range: {sm['SM1'].max()}, {sm['SM1'].min()}\nSM2 range: {sm['SM2'].max()}, {sm['SM2'].min()}\nSM3 range: {sm['SM3'].max()}, {sm['SM3'].min()}")
print(f"\nBattery Voltage range: {batt['Battery Voltage'].max()}, {batt['Battery Voltage'].min()}\nBattery Percentage range: {batt['Battery Percentage'].max()}, {batt['Battery Percentage'].min()}")
print(f"\nInternal Temperature range: {itemp['Internal Temperature'].max()}, {itemp['Internal Temperature'].min()}")
_ = input("Edit plotting and conversion parameters if necessary and press any key to continue...")

SM1 range: 21654.6, 9144.4
SM2 range: 21517.0, 9371.6
SM3 range: 21622.6, 9264.4

Battery Voltage range: 2.067327, 1.624521
Battery Percentage range: -125.8526, -175.0533

Internal Temperature range: -357.9128, -417.8635


In [5]:
ignore_nidx = 1
smn = ['SM1', 'SM2', 'SM3']
drop_sm = []
smx = list(set(smn) ^ set(drop_sm))

sm_raw_to_vwc = {
    'SM1': [22_000, 8_000],
    'SM2': [22_000, 8_000],
    'SM3': [22_000, 8_000]
}

batt_v_factor = 2
batt_p_factor = -1.4

In [6]:
batt

,Time,Battery Voltage,Battery Percentage
0,2024-07-17 06:00:00,1.761944,-159.7840
1,2024-07-19 14:30:00,1.739501,-162.2777
2,2024-07-19 14:30:00,1.741918,-162.0092
3,2024-07-19 14:45:00,1.747097,-161.4337
4,2024-07-19 15:00:00,1.714295,-165.0784
...,...,...,...
1375,2024-08-01 19:15:00,1.927141,-141.4288
1376,2024-08-01 19:30:00,1.926451,-141.5055
1377,2024-08-01 19:45:00,1.926105,-141.5439
1378,2024-08-01 20:00:00,1.925069,-141.6590


In [7]:
if drop_sm is not []:
    sm_filtered = sm[ignore_nidx:].copy().drop(columns=drop_sm)
else:
    sm_filtered = sm[ignore_nidx:].copy()

batt_filtered = batt[ignore_nidx:].copy().set_index('Time').reindex(sm_filtered['Time']).ffill().reset_index()
itemp_filtered = itemp[ignore_nidx:].copy().set_index('Time').reindex(sm_filtered['Time']).ffill().reset_index()

In [8]:
def convert_to_sm(raw_value, raw_dry, raw_sat):
    sm_value = (raw_value - raw_dry) / (raw_sat - raw_dry)
    return sm_value

station = sm_filtered.copy()
station['batt_v'] = batt_filtered['Battery Voltage']
station['batt_p'] = batt_filtered['Battery Percentage']
station['itemp'] = itemp_filtered['Internal Temperature']


for smidx in list(set(drop_sm) ^ set(smn)):
    station[[smidx]] = sm_filtered[[smidx]].map(lambda x: convert_to_sm(x, raw_dry=sm_raw_to_vwc[smidx][0], raw_sat=sm_raw_to_vwc[smidx][1]))

station['batt_v'] = station['batt_v'] * batt_v_factor
station['batt_p'] = station['batt_p'] * batt_p_factor

station['itemp'] = station['itemp'] + 273 + 25

cols = smx.copy()
cols.insert(0, 'Time')
station_plot_sm = station[cols].melt(id_vars=['Time'], value_name='SM', var_name='SMID')

In [9]:
fig = px.line(station_plot_sm, x='Time', y='SM', color='SMID', title=f'Station {stn.upper()}: Soil Moisture (VWC eqv., uncalibrated)')
fig.show()
fig.write_html(f'./vis/station_{stn}_sm.html')

In [10]:
fig = px.line(station, x='Time', y='batt_p', title=f'Station {stn.upper()}: Battery Percentage')
fig.update_traces(line=dict(color='cyan'))
fig.write_html(f'./vis/station_{stn}_batt.html')
fig.show()

In [11]:
fig = px.line(station, x='Time', y='itemp', title=f'Station {stn.upper()}: Internal Temperature')
fig.update_traces(line=dict(color='orange'))
fig.write_html(f'./vis/station_{stn}_itemp.html')
fig.show()